# <font color='#2F4F4F'>AfterWork Data Science: Market Basket Analysis- Project</font>

## <font color='#2F4F4F'>Step 1. Business Understanding </font>

### a) Specifying the Research Question
Perform market basket analysis to identify the top 10 products likely to be purchased together and help maximize revenue

### b) Defining the Metric for Success

 Determine which products have a strong association relationship

### c) Understanding the Context 
Care five is a German multinational retail corporation headquartered in Berlin, Germany. It is the eighth-largest retailer in the world by revenue. It operates a chain of hypermarkets, groceries stores, and convenience stores, which as of January 2021,comprises its 12,00 stores in over 30 countries.

As a Data analyst working for one of the stores, you must perform market basket
analysis to help the store maximize revenue. More specifically, your task will analyze
transactional data to identify the top 10 products likely to be purchased together.


### d) Recording the Experimental Design


● Perform data importation and loading

● Perform data preprocessing

● Find frequent itemsets

● Generate association rules

● Perform metric interpretation and provide recommendation


## <font color='#2F4F4F'>Step 2. Data Importation and Data Preprocessing</font>

In [95]:
# Importing standard libraries
# ---
# 
import pandas as pd              
import numpy as np               
import matplotlib.pyplot as plt 

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [96]:
# Importing our dataset
# ---
#
df=pd.read_csv("https://bit.ly/30A2gHO")

df.head(20)



,A,Quantity,Transaction,Store,Product
0,30000,2,93194,6,Magazine
1,30001,2,93194,6,Candy Bar
2,30002,2,93194,6,Candy Bar
3,30003,2,93194,6,Candy Bar
4,30004,2,93194,6,Candy Bar
5,30005,2,93197,1,Pencils
6,30006,1,93200,6,Candy Bar
7,30007,1,93200,6,Candy Bar
8,30008,1,93200,6,Candy Bar
9,30009,1,93200,6,Magazine


In [97]:
#Group the dataframe by Transaction, Product and store  
#  and display the count of items

df2 = df.groupby(["Transaction", "Product"]).size().reset_index(name="Count")
df2.head(20)

,Transaction,Product,Count
0,93194,Candy Bar,4
1,93194,Magazine,1
2,93197,Pencils,1
3,93200,Candy Bar,3
4,93200,Magazine,1
5,93206,Greeting Cards,1
6,93206,Magazine,1
7,93206,Pencils,2
8,93212,Toothbrush,1
9,93215,Candy Bar,2


In [98]:
#consolidate the items into one transaction per row 
# with each item one-hot encoded.

df3 = (df2.groupby(["Transaction", "Product"])['Count']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Transaction'))

df3.head()

Product,Bow,Candy Bar,Deodorant,Greeting Cards,Magazine,Markers,Pain Reliever,Pencils,Pens,Perfume,Photo Processing,Prescription Med,Shampoo,Soap,Toothbrush,Toothpaste,Wrapping Paper
Transaction,,,,,,,,,,,,,,,,,
93194,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93200,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93206,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [99]:
#use our custom encoding function to convert 
# all the values to 0 or 1. 
# The Apriori algorithm will only take 0's or 1's.

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

df4 = df3.applymap(encode_units)

df4

Product,Bow,Candy Bar,Deodorant,Greeting Cards,Magazine,Markers,Pain Reliever,Pencils,Pens,Perfume,Photo Processing,Prescription Med,Shampoo,Soap,Toothbrush,Toothpaste,Wrapping Paper
Transaction,,,,,,,,,,,,,,,,,
93194,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
93197,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
93200,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
93206,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
93212,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
133424,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
133427,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## <font color='#2F4F4F'>Step 3. Find frequent itemsets</font>

In [107]:
#generate the frequent itemsets
shop_frequent_itemsets = apriori(df4, min_support=0.01, use_colnames=True)

shop_frequent_itemsets

,support,itemsets
0,0.051591,(Bow)
1,0.175736,(Candy Bar)
2,0.152840,(Greeting Cards)
3,0.231936,(Magazine)
4,0.020071,(Pain Reliever)
5,0.135147,(Pencils)
6,0.144068,(Pens)
7,0.082664,(Perfume)
8,0.055456,(Photo Processing)
9,0.014422,(Prescription Med)


## <font color='#2F4F4F'>Step 4. Generate association rules</font>

In [108]:

# Finding the association rules
shop_rules = association_rules(shop_frequent_itemsets, metric="lift", min_threshold=1)


# Sorting 
shop_rules.sort_values("lift", ascending = False, inplace = True)

# Previewing the associative rules 


#top 10 combinations of items frequently bought together
shop_rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18,(Toothbrush),(Perfume),0.067648,0.082664,0.017098,0.252747,3.057514,0.011506,1.227611
19,(Perfume),(Toothbrush),0.082664,0.067648,0.017098,0.206835,3.057514,0.011506,1.175482
0,(Bow),(Toothbrush),0.051591,0.067648,0.010110,0.195965,2.896843,0.006620,1.159592
1,(Toothbrush),(Bow),0.067648,0.051591,0.010110,0.149451,2.896843,0.006620,1.115055
23,(Greeting Cards),"(Candy Bar, Magazine)",0.152840,0.039994,0.017247,0.112840,2.821431,0.011134,1.082112
22,"(Candy Bar, Magazine)",(Greeting Cards),0.039994,0.152840,0.017247,0.431227,2.821431,0.011134,1.489452
53,(Greeting Cards),"(Magazine, Pencils)",0.152840,0.028546,0.012043,0.078794,2.760244,0.007680,1.054546
52,"(Magazine, Pencils)",(Greeting Cards),0.028546,0.152840,0.012043,0.421875,2.760244,0.007680,1.465358
48,(Candy Bar),"(Toothpaste, Pencils)",0.175736,0.022748,0.011002,0.062606,2.752198,0.007005,1.042520
45,"(Toothpaste, Pencils)",(Candy Bar),0.022748,0.175736,0.011002,0.483660,2.752198,0.007005,1.596359


## <font color='#2F4F4F'>Step 5. Perform metric interpretation and provide recommendation</font>


- Toothbrush and Perfume are the items most likely to be purchased together as the highest lift 3.057514


- The confidence level for the 'Toothbrush and Perfume' rule is 0.252747, which shows that out of all the transactions that contain both ‘Toothbrush’, 25.2 percent contain ‘Perfume’ too.
 
-  Would recommend the business to place items likely to be bought together on similar sections and on bundled offers.
